In [19]:
import pandas as pd
from seaborn.external.docscrape import header


In [20]:
df1 = pd.read_csv('fintech1.csv')
df2 = pd.read_csv('fintech2.csv')

df = pd.concat([df1, df2])

In [21]:
df

,Publication Number,Publication Date,Publication Country Code,Application Number,Application Date,Priority Date - Earliest,Priority Date - Earliest - DWPI,Earliest Priority Year,Earliest Priority Year - DWPI,Application Country/Region,...,DWPI Count of Family Members,DWPI Count of Family Countries/Regions,Inventor - Original,Assignee - Original,Cited Patents - DPCI,Cited Patent Details - DPCI,Cited Non-patents - DPCI,Citing Patents - DPCI,Citing Patent Details - DPCI,Unnamed: 45
0,IN202441099749A,2024-12-20,IN,IN202441099749A,2024-12-17,2024-12-17,2024-12-17,2024,2024.0,IN,...,1.0,1.0,"Jayakani, S | Rani, Vennila Fathima S | Banu, ...",Jayakani S | Rani Vennila Fathima S | Banu Raf...,NaN,NaN,NaN,NaN,NaN,NaN
1,TR2024019326A2,2024-12-23,TR,TR202419326A,2024-12-16,2024-12-16,2024-12-16,2024,2024.0,TR,...,1.0,1.0,ÖMER YILMAZ,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,NaN,NaN,NaN,NaN,NaN,NaN
2,IN202441099497A,2024-12-27,IN,IN202441099497A,2024-12-16,2024-12-16,2024-12-16,2024,2024.0,IN,...,1.0,1.0,"Kanagaraj, Alias Ganesan M | Karthick, K K | V...",SAVEETHA ENGINEERING COLLEGE,NaN,NaN,NaN,NaN,NaN,NaN
3,TR2024018329A2,2024-12-23,TR,TR202418329A,2024-12-11,2024-12-11,2024-12-11,2024,2024.0,TR,...,1.0,1.0,ÖZGE GÜLER KUŞ | OZAN TEMMUZ GÜRCANOK,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,NaN,NaN,NaN,NaN,NaN,NaN
4,IN202441096888A,2024-12-13,IN,IN202441096888A,2024-12-08,2024-12-08,2024-12-08,2024,2024.0,IN,...,1.0,1.0,"Rai, Karunakara B | Hemalatha, K J | Pareek, P...",Rai Karunakara B | Hemalatha K J | Pareek Piyu...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7389,CA3037123C,2025-09-09,CA,CA3037123A,2017-08-08,2016-08-08,NaN,2016,NaN,CA,...,NaN,NaN,EOIN LANE | ANJANA BHATTACHARYYA | YUAN NIU | ...,THE DUN & BRADSTREET CORPORATION,NaN,NaN,NaN,NaN,NaN,NaN
7390,EP3507758B1,2025-09-17,EP,EP2017737105A,2017-06-21,2016-08-31,2016-08-31,2016,2016.0,EP,...,4.0,135.0,"MILLHUFF, Paul | LUSTYK, Neil | SCHEERER, John...",Chicago Mercantile Exchange Inc.,US20040205770A1 | US20050096999A1 | US20050203...,WO2018044377A1 | US20040205770A1 | A | 0 (Exam...,NaN,US11158001B2 | US20230206328A1,US9792164B1 | US11158001B2 | | 0 (Examiner) |...,NaN
7391,BR112018071839B1,2025-10-21,BR,BR11201871839A,2017-03-28,2016-04-25,NaN,2016,NaN,BR,...,NaN,NaN,"JONG HYUN WOO WOO, JONG HYUN",ESTORM CO. LTD.,NaN,NaN,NaN,NaN,NaN,NaN
7392,EP4525372A2,2025-03-19,EP,EP2025155185A,2016-02-12,2015-02-13,2015-02-13,2015,2015.0,EP,...,23.0,131.0,"LOUGHLIN-MCHUGH, Eleanor Simone Frederika | SZ...",Yoti Holding Limited,CN101641707A | EP1942430A1 | US20020165861A1 |...,US10210321B2 | CN101641707A | | 1 (Applicant)...,US10521623B2 | Elisa Bertino; Privacy-preservi...,AU2018333068B2 | CN107292622A | CN107292622B |...,US20160239653A1 | AU2018333068B2 | | | 20192...,NaN


In [22]:
df.shape

(33944, 46)

In [23]:
import pandas as pd
import numpy as np

# keep a copy to inspect failures later
orig = df['Application Date'].copy()

# normalize blanks
col = df['Application Date'].astype(str).str.strip().replace({'': pd.NA, 'nan': pd.NA})

# 1) parse mixed string formats
dt = pd.to_datetime(col, format='mixed', dayfirst=True, errors='coerce')

# 2) handle Excel serial numbers if present
mask_num = col.str.fullmatch(r'\d+')
serial = pd.to_numeric(col.where(mask_num), errors='coerce')
excel_dt = pd.to_datetime('1899-12-30') + pd.to_timedelta(serial, unit='D')
dt = dt.fillna(excel_dt)

# 3) drop anything still out of bounds or absurd years
dt = dt.where((dt >= pd.Timestamp('1677-09-21')) & (dt <= pd.Timestamp('2262-04-11')))
dt = dt.where(dt >= pd.Timestamp('1900-01-01'))  # tighten if you want

# assign back
df['Application Date'] = dt

# inspect rows that failed to parse
bad_rows = df[df['Application Date'].isna()][['Application Date']].assign(_raw=orig)
# print(bad_rows.head())  # optional

# your filter
df = df[(df['Application Date'] >= '2000-01-01') & (df['Application Date'] <= '2024-12-31')]

In [24]:
df.shape

(31519, 46)

In [25]:
df

,Publication Number,Publication Date,Publication Country Code,Application Number,Application Date,Priority Date - Earliest,Priority Date - Earliest - DWPI,Earliest Priority Year,Earliest Priority Year - DWPI,Application Country/Region,...,DWPI Count of Family Members,DWPI Count of Family Countries/Regions,Inventor - Original,Assignee - Original,Cited Patents - DPCI,Cited Patent Details - DPCI,Cited Non-patents - DPCI,Citing Patents - DPCI,Citing Patent Details - DPCI,Unnamed: 45
0,IN202441099749A,2024-12-20,IN,IN202441099749A,2024-12-17,2024-12-17,2024-12-17,2024,2024.0,IN,...,1.0,1.0,"Jayakani, S | Rani, Vennila Fathima S | Banu, ...",Jayakani S | Rani Vennila Fathima S | Banu Raf...,NaN,NaN,NaN,NaN,NaN,NaN
1,TR2024019326A2,2024-12-23,TR,TR202419326A,2024-12-16,2024-12-16,2024-12-16,2024,2024.0,TR,...,1.0,1.0,ÖMER YILMAZ,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,NaN,NaN,NaN,NaN,NaN,NaN
2,IN202441099497A,2024-12-27,IN,IN202441099497A,2024-12-16,2024-12-16,2024-12-16,2024,2024.0,IN,...,1.0,1.0,"Kanagaraj, Alias Ganesan M | Karthick, K K | V...",SAVEETHA ENGINEERING COLLEGE,NaN,NaN,NaN,NaN,NaN,NaN
3,TR2024018329A2,2024-12-23,TR,TR202418329A,2024-12-11,2024-12-11,2024-12-11,2024,2024.0,TR,...,1.0,1.0,ÖZGE GÜLER KUŞ | OZAN TEMMUZ GÜRCANOK,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,NaN,NaN,NaN,NaN,NaN,NaN
4,IN202441096888A,2024-12-13,IN,IN202441096888A,2024-12-08,2024-12-08,2024-12-08,2024,2024.0,IN,...,1.0,1.0,"Rai, Karunakara B | Hemalatha, K J | Pareek, P...",Rai Karunakara B | Hemalatha K J | Pareek Piyu...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7389,CA3037123C,2025-09-09,CA,CA3037123A,2017-08-08,2016-08-08,NaN,2016,NaN,CA,...,NaN,NaN,EOIN LANE | ANJANA BHATTACHARYYA | YUAN NIU | ...,THE DUN & BRADSTREET CORPORATION,NaN,NaN,NaN,NaN,NaN,NaN
7390,EP3507758B1,2025-09-17,EP,EP2017737105A,2017-06-21,2016-08-31,2016-08-31,2016,2016.0,EP,...,4.0,135.0,"MILLHUFF, Paul | LUSTYK, Neil | SCHEERER, John...",Chicago Mercantile Exchange Inc.,US20040205770A1 | US20050096999A1 | US20050203...,WO2018044377A1 | US20040205770A1 | A | 0 (Exam...,NaN,US11158001B2 | US20230206328A1,US9792164B1 | US11158001B2 | | 0 (Examiner) |...,NaN
7391,BR112018071839B1,2025-10-21,BR,BR11201871839A,2017-03-28,2016-04-25,NaN,2016,NaN,BR,...,NaN,NaN,"JONG HYUN WOO WOO, JONG HYUN",ESTORM CO. LTD.,NaN,NaN,NaN,NaN,NaN,NaN
7392,EP4525372A2,2025-03-19,EP,EP2025155185A,2016-02-12,2015-02-13,2015-02-13,2015,2015.0,EP,...,23.0,131.0,"LOUGHLIN-MCHUGH, Eleanor Simone Frederika | SZ...",Yoti Holding Limited,CN101641707A | EP1942430A1 | US20020165861A1 |...,US10210321B2 | CN101641707A | | 1 (Applicant)...,US10521623B2 | Elisa Bertino; Privacy-preservi...,AU2018333068B2 | CN107292622A | CN107292622B |...,US20160239653A1 | AU2018333068B2 | | | 20192...,NaN


In [26]:
df = df.copy()  # safe even if not a view

year = pd.to_numeric(df['Earliest Priority Year'], errors='coerce')
current = pd.Timestamp.today().year

df.loc[:, 'Legal Years Remaining'] = (20 - (current - year)).clip(lower=0)

In [27]:
df

,Publication Number,Publication Date,Publication Country Code,Application Number,Application Date,Priority Date - Earliest,Priority Date - Earliest - DWPI,Earliest Priority Year,Earliest Priority Year - DWPI,Application Country/Region,...,DWPI Count of Family Countries/Regions,Inventor - Original,Assignee - Original,Cited Patents - DPCI,Cited Patent Details - DPCI,Cited Non-patents - DPCI,Citing Patents - DPCI,Citing Patent Details - DPCI,Unnamed: 45,Legal Years Remaining
0,IN202441099749A,2024-12-20,IN,IN202441099749A,2024-12-17,2024-12-17,2024-12-17,2024,2024.0,IN,...,1.0,"Jayakani, S | Rani, Vennila Fathima S | Banu, ...",Jayakani S | Rani Vennila Fathima S | Banu Raf...,NaN,NaN,NaN,NaN,NaN,NaN,19
1,TR2024019326A2,2024-12-23,TR,TR202419326A,2024-12-16,2024-12-16,2024-12-16,2024,2024.0,TR,...,1.0,ÖMER YILMAZ,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,NaN,NaN,NaN,NaN,NaN,NaN,19
2,IN202441099497A,2024-12-27,IN,IN202441099497A,2024-12-16,2024-12-16,2024-12-16,2024,2024.0,IN,...,1.0,"Kanagaraj, Alias Ganesan M | Karthick, K K | V...",SAVEETHA ENGINEERING COLLEGE,NaN,NaN,NaN,NaN,NaN,NaN,19
3,TR2024018329A2,2024-12-23,TR,TR202418329A,2024-12-11,2024-12-11,2024-12-11,2024,2024.0,TR,...,1.0,ÖZGE GÜLER KUŞ | OZAN TEMMUZ GÜRCANOK,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,NaN,NaN,NaN,NaN,NaN,NaN,19
4,IN202441096888A,2024-12-13,IN,IN202441096888A,2024-12-08,2024-12-08,2024-12-08,2024,2024.0,IN,...,1.0,"Rai, Karunakara B | Hemalatha, K J | Pareek, P...",Rai Karunakara B | Hemalatha K J | Pareek Piyu...,NaN,NaN,NaN,NaN,NaN,NaN,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7389,CA3037123C,2025-09-09,CA,CA3037123A,2017-08-08,2016-08-08,NaN,2016,NaN,CA,...,NaN,EOIN LANE | ANJANA BHATTACHARYYA | YUAN NIU | ...,THE DUN & BRADSTREET CORPORATION,NaN,NaN,NaN,NaN,NaN,NaN,11
7390,EP3507758B1,2025-09-17,EP,EP2017737105A,2017-06-21,2016-08-31,2016-08-31,2016,2016.0,EP,...,135.0,"MILLHUFF, Paul | LUSTYK, Neil | SCHEERER, John...",Chicago Mercantile Exchange Inc.,US20040205770A1 | US20050096999A1 | US20050203...,WO2018044377A1 | US20040205770A1 | A | 0 (Exam...,NaN,US11158001B2 | US20230206328A1,US9792164B1 | US11158001B2 | | 0 (Examiner) |...,NaN,11
7391,BR112018071839B1,2025-10-21,BR,BR11201871839A,2017-03-28,2016-04-25,NaN,2016,NaN,BR,...,NaN,"JONG HYUN WOO WOO, JONG HYUN",ESTORM CO. LTD.,NaN,NaN,NaN,NaN,NaN,NaN,11
7392,EP4525372A2,2025-03-19,EP,EP2025155185A,2016-02-12,2015-02-13,2015-02-13,2015,2015.0,EP,...,131.0,"LOUGHLIN-MCHUGH, Eleanor Simone Frederika | SZ...",Yoti Holding Limited,CN101641707A | EP1942430A1 | US20020165861A1 |...,US10210321B2 | CN101641707A | | 1 (Applicant)...,US10521623B2 | Elisa Bertino; Privacy-preservi...,AU2018333068B2 | CN107292622A | CN107292622B |...,US20160239653A1 | AU2018333068B2 | | | 20192...,NaN,10


In [28]:
#to get count of IPC Sections

import re

def normalize(entry: str) -> str:
    # lowercase
    entry = entry.lower()
    # replace hyphens with spaces
    entry = entry.replace('-', ' ')
    # remove punctuation except spaces/letters/numbers
    entry = re.sub(r'[^\w\s]', '', entry)
    # collapse multiple spaces
    entry = re.sub(r'\s+', ' ', entry)
    # trim
    return entry.strip()

def count_unique_entries(text_blob: str) -> int:
    if pd.isna(text_blob):
        return 0
    if not text_blob:
        return 0
    entries = [normalize(entry) for entry in text_blob.split(',')]
    return len(set(entries))


In [29]:
df['IPC Count'] = df['IPC Subclass'].apply(count_unique_entries)

In [30]:
## To get Unique and standardize assignees

import re

def normalize(entry: str) -> str:
    if not isinstance(entry, str):
        return ""
    entry = entry.upper()
    entry = entry.replace('-', ' ')
    entry = re.sub(r'[^\w\s]', '', entry)
    entry = re.sub(r'\s+', ' ', entry)
    return entry.strip()

def unique_entries(text_blob) -> str:
    if not isinstance(text_blob, str) or not text_blob.strip():
        return ""
    entries = [normalize(entry) for entry in text_blob.split('|')]
    unique = list(dict.fromkeys(entries))  # keeps order
    return " | ".join([e for e in unique if e])  # drop empty strings

In [31]:
df['Assignee Standardized'] = df['Assignee - Original'].apply(unique_entries)
df['Assignee Standardized']

0       JAYAKANI S | RANI VENNILA FATHIMA S | BANU RAF...
1       TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...
2                            SAVEETHA ENGINEERING COLLEGE
3       TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...
4       RAI KARUNAKARA B | HEMALATHA K J | PAREEK PIYU...
                              ...                        
7389                       THE DUN BRADSTREET CORPORATION
7390                      CHICAGO MERCANTILE EXCHANGE INC
7391                                        ESTORM CO LTD
7392                                 YOTI HOLDING LIMITED
7393                                 YOTI HOLDING LIMITED
Name: Assignee Standardized, Length: 31519, dtype: object

In [32]:
df['Inventor Standardized'] = df['Inventor - Original'].apply(unique_entries)
df['Inventor Standardized']

0       JAYAKANI S | RANI VENNILA FATHIMA S | BANU RAF...
1                                             ÖMER YILMAZ
2       KANAGARAJ ALIAS GANESAN M | KARTHICK K K | VIM...
3                   ÖZGE GÜLER KUŞ | OZAN TEMMUZ GÜRCANOK
4       RAI KARUNAKARA B | HEMALATHA K J | PAREEK PIYU...
                              ...                        
7389    EOIN LANE | ANJANA BHATTACHARYYA | YUAN NIU | ...
7390    MILLHUFF PAUL | LUSTYK NEIL | SCHEERER JOHN | ...
7391                          JONG HYUN WOO WOO JONG HYUN
7392    LOUGHLIN MCHUGH ELEANOR SIMONE FREDERIKA | SZC...
7393    LOUGHLIN MCHUGH ELEANOR SIMONE FREDERIKA | SZC...
Name: Inventor Standardized, Length: 31519, dtype: object

In [33]:
cols = ['Publication Number','Application Date','Publication Date','IPC Subclass', 'Assignee - Original','Assignee Standardized','Claims Count','Assignee Count', 'Inventor - Original','Inventor Standardized','Publication Country Code','Count of Cited Refs - Patent','Inventor Count', 'IPC Count','Count of Cited Refs - Non-patent','Count of Citing Patents','DWPI Count of Family Members','DWPI Count of Family Countries/Regions', 'Legal Years Remaining']
df=df[cols]

In [34]:
#Now delete all the columns for which all rows are having value null
df = df.dropna(axis=1, how='all')
print('data shape=',df.shape)

data shape= (31519, 19)


In [35]:
df.isnull().sum()

Publication Number                          0
Application Date                            0
Publication Date                            0
IPC Subclass                              257
Assignee - Original                       160
Assignee Standardized                       0
Claims Count                               64
Assignee Count                              0
Inventor - Original                       434
Inventor Standardized                       0
Publication Country Code                    0
Count of Cited Refs - Patent                0
Inventor Count                              0
IPC Count                                   0
Count of Cited Refs - Non-patent            0
Count of Citing Patents                     0
DWPI Count of Family Members              191
DWPI Count of Family Countries/Regions    191
Legal Years Remaining                       0
dtype: int64

In [36]:
INDICATORS = [
    "Count of Cited Refs - Patent",
    "Count of Cited Refs - Non-patent",
    "Count of Citing Patents",
    "DWPI Count of Family Members",
    "DWPI Count of Family Countries/Regions",
    "Assignee Count",
    "Assignee - Original",
    "Inventor - Original",
    "Inventor Count",
    "Claims Count",
    "IPC Subclass",
    "Assignee Standardized",
    "Inventor Standardized",
    "IPC Count",
]

In [37]:
df = df[df[INDICATORS].notna().all(axis=1)]


In [38]:
df.shape


(30592, 19)

In [39]:
df.to_csv("MainData.csv", index=False)


In [40]:
df

,Publication Number,Application Date,Publication Date,IPC Subclass,Assignee - Original,Assignee Standardized,Claims Count,Assignee Count,Inventor - Original,Inventor Standardized,Publication Country Code,Count of Cited Refs - Patent,Inventor Count,IPC Count,Count of Cited Refs - Non-patent,Count of Citing Patents,DWPI Count of Family Members,DWPI Count of Family Countries/Regions,Legal Years Remaining
0,IN202441099749A,2024-12-17,2024-12-20,H04L | G06Q,Jayakani S | Rani Vennila Fathima S | Banu Raf...,JAYAKANI S | RANI VENNILA FATHIMA S | BANU RAF...,10.0,5,"Jayakani, S | Rani, Vennila Fathima S | Banu, ...",JAYAKANI S | RANI VENNILA FATHIMA S | BANU RAF...,IN,0,5,1,0,0,1.0,1.0,19
1,TR2024019326A2,2024-12-16,2024-12-23,G06Q,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,16.0,1,ÖMER YILMAZ,ÖMER YILMAZ,TR,0,1,1,0,0,1.0,1.0,19
2,IN202441099497A,2024-12-16,2024-12-27,G06Q,SAVEETHA ENGINEERING COLLEGE,SAVEETHA ENGINEERING COLLEGE,6.0,1,"Kanagaraj, Alias Ganesan M | Karthick, K K | V...",KANAGARAJ ALIAS GANESAN M | KARTHICK K K | VIM...,IN,0,4,1,0,0,1.0,1.0,19
3,TR2024018329A2,2024-12-11,2024-12-23,G06Q,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,TURKCELL TEKNOLOJİ ARAŞTIRMA VE GELİŞTİRME ANO...,8.0,2,ÖZGE GÜLER KUŞ | OZAN TEMMUZ GÜRCANOK,ÖZGE GÜLER KUŞ | OZAN TEMMUZ GÜRCANOK,TR,0,2,1,0,0,1.0,1.0,19
4,IN202441096888A,2024-12-08,2024-12-13,G06Q | G06N | B32B,Rai Karunakara B | Hemalatha K J | Pareek Piyu...,RAI KARUNAKARA B | HEMALATHA K J | PAREEK PIYU...,3.0,4,"Rai, Karunakara B | Hemalatha, K J | Pareek, P...",RAI KARUNAKARA B | HEMALATHA K J | PAREEK PIYU...,IN,0,3,1,0,0,1.0,1.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,EP3667595B1,2018-08-07,2025-02-26,G06Q,NTI Inc.,NTI INC,2.0,1,"NAKAMURA, Takatoshi",NAKAMURA TAKATOSHI,EP,7,1,1,6,0,8.0,135.0,12
7388,EP3627420B1,2018-05-17,2025-07-09,H04L | G06Q,NTI Inc.,NTI INC,2.0,1,"NAKAMURA, Takatoshi",NAKAMURA TAKATOSHI,EP,10,1,1,1,0,11.0,135.0,12
7390,EP3507758B1,2017-06-21,2025-09-17,G06Q,Chicago Mercantile Exchange Inc.,CHICAGO MERCANTILE EXCHANGE INC,15.0,1,"MILLHUFF, Paul | LUSTYK, Neil | SCHEERER, John...",MILLHUFF PAUL | LUSTYK NEIL | SCHEERER JOHN | ...,EP,5,5,1,0,0,4.0,135.0,11
7392,EP4525372A2,2016-02-12,2025-03-19,H04L,Yoti Holding Limited,YOTI HOLDING LIMITED,15.0,1,"LOUGHLIN-MCHUGH, Eleanor Simone Frederika | SZ...",LOUGHLIN MCHUGH ELEANOR SIMONE FREDERIKA | SZC...,EP,2,2,1,0,0,23.0,131.0,10
